### Load app_store_id from top_chart

In [1]:
import pandas as pd
from pandas_gbq import read_gbq
import logging
import google.cloud.logging
from google.cloud.logging.handlers import CloudLoggingHandler

# Initialize the Stackdriver client
client = google.cloud.logging.Client()

# Create a Cloud Logging handler for Stackdriver
cloud_handler = CloudLoggingHandler(client)

# Set up the logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)  # Set to INFO for standard logging, DEBUG for more detailed logs

# Add Cloud Logging and Stream handlers to the logger
stream_handler = logging.StreamHandler()
logger.addHandler(cloud_handler)  # Sends logs to Stackdriver
logger.addHandler(stream_handler)  # Sends logs to console

# Optional: Set the logging format for console output (the Cloud handler already formats for Stackdriver)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Define your LIMIT and OFFSET
limit = 100
offset = 2000

# Define your BigQuery SQL query
query = f"""
    SELECT * FROM `web-scraping-2024.top_chart.top_chart`
    LIMIT {limit} OFFSET {offset}
"""

# Load the data into a Pandas DataFrame
df = read_gbq(query, project_id='web-scraping-2024')

# Generate metadata
metadata = df['app_store_id'].apply(lambda x: f"https://apps.apple.com/US/app/id{x}")

# Define messages to log and print
rows_info = f"Retrieved rows {offset + 1} to {offset + len(df)} out of total rows in the dataset."
data_shape_info = f"Current DataFrame shape: {df.shape}"

# Log and print the current position and size of the DataFrame
logger.info(rows_info)   # Logs to both Stackdriver and console
print(rows_info)         # Prints to console

logger.info(data_shape_info)  # Logs to both Stackdriver and console
print(data_shape_info)        # Prints to console

# Log the first few rows of the DataFrame at the debug level
preview_info = df.head()
logger.debug(f"DataFrame preview:\n{preview_info}")  # This log will only appear if logging level is DEBUG


Downloading: 100%|██████████|

2024-09-26 12:55:09,819 - INFO - Retrieved rows 2001 to 2100 out of total rows in the dataset.
INFO:__main__:Retrieved rows 2001 to 2100 out of total rows in the dataset.
2024-09-26 12:55:09,841 - INFO - Current DataFrame shape: (100, 12)
INFO:__main__:Current DataFrame shape: (100, 12)



Retrieved rows 2001 to 2100 out of total rows in the dataset.
Current DataFrame shape: (100, 12)


### Scrape Data

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import re
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to fetch data from the app store page
def fetch_app_store_data(app_store_url):
    try:
        # Send a request to the app store URL
        response = requests.get(app_store_url, allow_redirects=True)

        # If redirection occurs, get the final URL
        scraping_url_redirect = response.url

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the required details
        title_with_rating = soup.find('h1', class_='product-header__title app-header__title').text.strip() if soup.find('h1', class_='product-header__title app-header__title') else None

        # Remove age rating (like "12+") from title
        title = re.sub(r'\s*\d+\+\s*$', '', title_with_rating) if title_with_rating else None

        subtitle = soup.find('h2', class_='product-header__subtitle app-header__subtitle').text.strip() if soup.find('h2', class_='product-header__subtitle app-header__subtitle') else None
        avg_rating = soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[0] if soup.find('figcaption', class_='we-rating-count star-rating__count') else None

        # Extract number of ratings and remove the word "Ratings"
        number_of_ratings = (
            soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[1].replace(' Ratings', '')
            if soup.find('figcaption', class_='we-rating-count star-rating__count') else None
        )

        developer_name = soup.find('dd', class_='information-list__item__definition').text.strip() if soup.find('dd', class_='information-list__item__definition') else None
        developer_url = soup.find('h2', class_='product-header__identity app-header__identity').find('a')['href'] if soup.find('h2', class_='product-header__identity app-header__identity') else None

        # Extract size from the specific tag by finding the 'Size' <dt>
        size = None
        size_term = soup.find('dt', string='Size')  # Use 'string' instead of 'text'
        if size_term:
            size_definition = size_term.find_next_sibling('dd')  # Get the corresponding <dd>
            size = size_definition.text.strip() if size_definition else None

        data = {
            "scraping_url": app_store_url,
            "scraping_timestamp": int(datetime.now().timestamp() * 1_000_000),
            "scraping_url_redirect": scraping_url_redirect,
            "title": title,
            "subtitle": subtitle,
            "avg_rating": float(avg_rating) if avg_rating else None,
            "number_of_ratings": number_of_ratings,  # Only the numeric part
            "developer_name": developer_name,
            "developer_url": developer_url,
            "size": size
        }

        message = f"Successfully fetched data for {app_store_url}"
        logger.info(message)

        return data

    except Exception as e:
        error_message = f"Error fetching data for {app_store_url}: {e}"
        logger.error(error_message)
        return None

# Example usage:
# Assuming you have a DataFrame df with the app_store_url and other relevant columns.
scraped_data = []
for index, row in df.iterrows():
    app_store_url = f"https://apps.apple.com/us/app/id{row['app_store_id']}"  # Assuming you build the URL like this

    # Fetch app details
    app_data = fetch_app_store_data(app_store_url)

    if app_data:
        scraped_data.append(app_data)

    # Sleep to avoid overwhelming the server
    time.sleep(1)

# Create a DataFrame from the scraped data
df_scraped = pd.DataFrame(scraped_data)

# Ensure columns are in the specified order
df_scraped = df_scraped[[
    "scraping_url",
    "scraping_timestamp",
    "scraping_url_redirect",
    "title",
    "subtitle",
    "avg_rating",
    "number_of_ratings",
    "developer_name",
    "developer_url",
    "size"
]]


2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched data for https://apps.apple.com/us/app/id722784828
2024-09-25 16:34:48,422 - INFO - Successfully fetched d

## Upload on GCS

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
import re
from google.cloud import storage
import io
import logging

# Set up logging for StackDriver (or other destinations)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Initialize GCS client
client = storage.Client()

# Define your bucket name
bucket_name = 'app_metadata_top_chart'
bucket = client.bucket(bucket_name)

# Function to upload Parquet file to GCS
def upload_parquet_to_gcs(df, file_name, bucket):
    try:
        # Create an in-memory bytes buffer to save the parquet file
        buffer = io.BytesIO()

        # Write the DataFrame to Parquet format in memory
        df.to_parquet(buffer, index=False)

        # Move the buffer's position to the beginning
        buffer.seek(0)

        # Create a blob (file object) in the bucket
        blob = bucket.blob(file_name)

        # Upload the bytes from the buffer to GCS
        blob.upload_from_file(buffer, content_type='application/octet-stream')

        message = f"Uploaded {file_name} to GCS bucket {bucket_name}"
        logger.info(message)
        print(message)
    except Exception as e:
        error_message = f"Failed to upload {file_name} to GCS: {e}"
        logger.error(error_message)
        print(error_message)

# Function to fetch data from the app store page
def fetch_app_store_data(app_store_url):
    try:
        # Send a request to the app store URL
        response = requests.get(app_store_url, allow_redirects=True)

        # If redirection occurs, get the final URL
        scraping_url_redirect = response.url

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the required details
        title_with_rating = soup.find('h1', class_='product-header__title app-header__title').text.strip() if soup.find('h1', class_='product-header__title app-header__title') else None

        # Remove age rating (like "12+") from title
        title = re.sub(r'\s*\d+\+\s*$', '', title_with_rating) if title_with_rating else None

        subtitle = soup.find('h2', class_='product-header__subtitle app-header__subtitle').text.strip() if soup.find('h2', class_='product-header__subtitle app-header__subtitle') else None
        avg_rating = soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[0] if soup.find('figcaption', class_='we-rating-count star-rating__count') else None

        # Extract number of ratings and remove the word "Ratings"
        number_of_ratings = (
            soup.find('figcaption', class_='we-rating-count star-rating__count').text.strip().split(' • ')[1].replace(' Ratings', '')
            if soup.find('figcaption', class_='we-rating-count star-rating__count') else None
        )

        developer_name = soup.find('dd', class_='information-list__item__definition').text.strip() if soup.find('dd', class_='information-list__item__definition') else None
        developer_url = soup.find('h2', class_='product-header__identity app-header__identity').find('a')['href'] if soup.find('h2', class_='product-header__identity app-header__identity') else None

        # Extract size from the specific tag by finding the 'Size' <dt>
        size = None
        size_term = soup.find('dt', string='Size')  # Use 'string' instead of 'text'
        if size_term:
            size_definition = size_term.find_next_sibling('dd')  # Get the corresponding <dd>
            size = size_definition.text.strip() if size_definition else None

        return {
            "scraping_url": app_store_url,
            "scraping_timestamp": int(datetime.now().timestamp() * 1_000_000),
            "scraping_url_redirect": scraping_url_redirect,
            "title": title,
            "subtitle": subtitle,
            "avg_rating": float(avg_rating) if avg_rating else None,
            "number_of_ratings": number_of_ratings,  # Only the numeric part
            "developer_name": developer_name,
            "developer_url": developer_url,
            "size": size
        }
    except Exception as e:
        error_message = f"Error fetching data for {app_store_url}: {e}"
        logger.error(error_message)
        print(error_message)
        return None

# Assuming you have a DataFrame df with the app_store_id and other relevant columns.
scraped_data = []
for index, row in df.iterrows():
    app_store_url = f"https://apps.apple.com/us/app/id{row['app_store_id']}"  # Assuming you build the URL like this

    # Fetch app details
    app_data = fetch_app_store_data(app_store_url)

    if app_data:
        # Append to the list
        scraped_data.append(app_data)

        # Create a DataFrame for the single app data
        df_app = pd.DataFrame([app_data])

        # Define the file path using app_store_id and scraping date
        app_id = row['app_store_id']
        scraping_date = datetime.now().strftime('%Y_%m_%d')
        file_name = f"{scraping_date}__{app_id}.parquet"

        # Upload the DataFrame to GCS
        upload_parquet_to_gcs(df_app, file_name, bucket)

    # Sleep to avoid overwhelming the server
    time.sleep(1)

# Optionally, create a DataFrame from all scraped data if needed
if scraped_data:
    df_scraped = pd.DataFrame(scraped_data)

    # Ensure columns are in the specified order
    df_scraped = df_scraped[[
        "scraping_url",
        "scraping_timestamp",
        "scraping_url_redirect",
        "title",
        "subtitle",
        "avg_rating",
        "number_of_ratings",
        "developer_name",
        "developer_url",
        "size"
    ]]


2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26,640 - INFO - Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:26

Uploaded 2024_09_25__722784828.parquet to GCS bucket app_metadata_top_chart


2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29,396 - INFO - Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:29

Uploaded 2024_09_25__395135620.parquet to GCS bucket app_metadata_top_chart


2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31,720 - INFO - Uploaded 2024_09_25__779848086.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:39:31

...

Uploaded 2024_09_25__648624928.parquet to GCS bucket app_metadata_top_chart


2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20,567 - INFO - Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:20

Uploaded 2024_09_25__919442604.parquet to GCS bucket app_metadata_top_chart


2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22,188 - INFO - Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart
2024-09-25 16:42:22

Uploaded 2024_09_25__582853306.parquet to GCS bucket app_metadata_top_chart


In [ ]:
df_scraped.head()

,scraping_url,scraping_timestamp,scraping_url_redirect,title,subtitle,avg_rating,number_of_ratings,developer_name,developer_url,size
0,https://apps.apple.com/us/app/id722784828,1727282365797991,https://apps.apple.com/us/app/the-bedtime-expr...,The Bedtime Express : The bedtime story that c...,None,4.7,19,Matt Robb,https://apps.apple.com/us/developer/unit11/id4...,104.3 MB
1,https://apps.apple.com/us/app/id395135620,1727282368439431,https://apps.apple.com/us/app/esh-mishna-%D7%9...,Esh Mishna אש משנ‪ה‬,None,5.0,1 Rating,Elyahu Sheetrit,https://apps.apple.com/us/developer/elyahu-she...,55.1 MB
2,https://apps.apple.com/us/app/id779848086,1727282370599443,https://apps.apple.com/us/app/berenstain-bears...,Berenstain Bears' Dinosaur Dig,Berenstain Bears kid's book,4.9,16,"Oceanhouse Media, Inc.",https://apps.apple.com/us/developer/oceanhouse...,45.8 MB
3,https://apps.apple.com/us/app/id867822295,1727282372908412,https://apps.apple.com/us/app/ccchymns/id86782...,CCCHymns,Enjoy Hymn in English & Yoruba,3.6,5,Adesegun Adesegun,https://apps.apple.com/us/developer/adesegun-a...,28.2 MB
4,https://apps.apple.com/us/app/id659683137,1727282374937199,https://apps.apple.com/us/app/byron-barton-col...,Byron Barton Collection #1,Four Classic Book apps in one‪!‬,4.3,69,"Oceanhouse Media, Inc.",https://apps.apple.com/us/developer/oceanhouse...,67 MB


### Load on BQ

In [5]:
import logging
from google.cloud import bigquery, storage
import pandas as pd

# Initialize clients
gcs_client = storage.Client()
bq_client = bigquery.Client()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define your parameters
project_id = "web-scraping-2024"
dataset_id = "app_metadata"
bucket_name = "app_metadata_top_chart"
app_metadata_dir_prefix = ""

def list_parquet_files(bucket_name, prefix):
    """List all parquet files in a GCS bucket with a given prefix."""
    bucket = gcs_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.parquet')]

def update_table_schema_to_string(table_ref):
    """Update all fields in the table schema to STRING."""
    table = bq_client.get_table(table_ref)
    logger.info(f"Current schema: {table.schema}")

    new_schema = []
    for field in table.schema:
        new_schema.append(bigquery.SchemaField(field.name, "STRING"))  # Change all fields to STRING

    # Update the table with the modified schema
    table.schema = new_schema
    updated_table = bq_client.update_table(table, ["schema"])  # Update only the schema
    logger.info(f"Updated table {updated_table.project}.{updated_table.dataset_id}.{updated_table.table_id} schema to STRING.")

# Load all parquet files
app_metadata_details_files = list_parquet_files(bucket_name, app_metadata_dir_prefix)

if not app_metadata_details_files:
    logger.warning("No parquet files found.")
else:
    app_metadata_table_ref = bq_client.dataset(dataset_id).table("app_metadata")

    # Update the schema to modify all fields to STRING
    update_table_schema_to_string(app_metadata_table_ref)

    app_metadata_job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,  # Parquet file format
        autodetect=True
    )

    for file_name in app_metadata_details_files:
        logger.info(f"Starting load job for file: {file_name}")

        # Load the Parquet file into a DataFrame to convert all data to STRING
        df = pd.read_parquet(f"gs://{bucket_name}/{file_name}")

        # Convert all columns to STRING
        df = df.astype(str)

        # Save the modified DataFrame back to a temporary Parquet file
        temp_parquet_path = f"gs://{bucket_name}/temp_{file_name}"
        df.to_parquet(temp_parquet_path)

        # Start the load job for each file
        load_job = bq_client.load_table_from_uri(
            temp_parquet_path,
            app_metadata_table_ref,
            job_config=app_metadata_job_config
        )

        # Wait for the job to complete
        load_job.result()

        # Check the result for each file
        app_metadata_details_table = bq_client.get_table(app_metadata_table_ref)
        logger.info(f"Loaded {app_metadata_details_table.num_rows} rows from {file_name} into {dataset_id}:app_metadata.")

        # Clean up: delete the temporary Parquet file if needed (optional)
        # gcs_client.bucket(bucket_name).delete_blob(f"temp_{file_name}")

logger.info("All files processed successfully.")


2024-09-26 13:01:21,276 - INFO - Current schema: []
INFO:__main__:Current schema: []
2024-09-26 13:01:21,627 - INFO - Updated table web-scraping-2024.app_metadata.app_metadata schema to STRING.
INFO:__main__:Updated table web-scraping-2024.app_metadata.app_metadata schema to STRING.
2024-09-26 13:01:21,631 - INFO - Starting load job for file: 2024_09_25__1000897885.parquet
INFO:__main__:Starting load job for file: 2024_09_25__1000897885.parquet
2024-09-26 13:01:29,456 - INFO - Loaded 1 rows from 2024_09_25__1000897885.parquet into app_metadata:app_metadata.
INFO:__main__:Loaded 1 rows from 2024_09_25__1000897885.parquet into app_metadata:app_metadata.
2024-09-26 13:01:29,461 - INFO - Starting load job for file: 2024_09_25__1010311475.parquet
INFO:__main__:Starting load job for file: 2024_09_25__1010311475.parquet
2024-09-26 13:01:35,772 - INFO - Loaded 2 rows from 2024_09_25__1010311475.parquet into app_metadata:app_metadata.
INFO:__main__:Loaded 2 rows from 2024_09_25__1010311475.parq

KeyboardInterrupt: 

In [7]:
import logging
from google.cloud import bigquery, storage
import pandas as pd

# Initialize clients
gcs_client = storage.Client()
bq_client = bigquery.Client()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define your parameters
project_id = "web-scraping-2024"
dataset_id = "app_metadata"
bucket_name = "app_metadata_top_chart"
app_metadata_dir_prefix = ""

def list_parquet_files(bucket_name, prefix):
    """List all parquet files in a GCS bucket with a given prefix."""
    bucket = gcs_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.parquet')]

def update_table_schema(table_ref):
    """Update the table schema to STRING type for all fields."""
    table = bq_client.get_table(table_ref)
    logger.info(f"Current schema: {table.schema}")

    new_schema = []
    for field in table.schema:
        new_schema.append(bigquery.SchemaField(field.name, "STRING"))  # Change all fields to STRING

    # Update the table with the modified schema
    table.schema = new_schema
    updated_table = bq_client.update_table(table, ["schema"])  # Update only the schema
    logger.info(f"Updated table {updated_table.project}.{updated_table.dataset_id}.{updated_table.table_id} schema to STRING.")

# Load all parquet files
app_metadata_details_files = list_parquet_files(bucket_name, app_metadata_dir_prefix)

if not app_metadata_details_files:
    logger.warning("No parquet files found.")
else:
    app_metadata_table_ref = bq_client.dataset(dataset_id).table("app_metadata")

    # Update the schema to modify all fields to STRING
    update_table_schema(app_metadata_table_ref)

    app_metadata_job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,  # Parquet file format
        autodetect=True
    )

    for file_name in app_metadata_details_files:
        logger.info(f"Starting load job for file: {file_name}")

        # Load the Parquet file into a DataFrame
        df = pd.read_parquet(f"gs://{bucket_name}/{file_name}")

        # Explicitly cast all columns to STRING using str(x)
        for column in df.columns:
            df[column] = df[column].apply(lambda x: str(x) if pd.notnull(x) else '')

        # Save the modified DataFrame back to a temporary Parquet file
        temp_parquet_path = f"gs://{bucket_name}/temp_{file_name}"
        df.to_parquet(temp_parquet_path)

        # Start the load job for each file
        load_job = bq_client.load_table_from_uri(
            temp_parquet_path,
            app_metadata_table_ref,
            job_config=app_metadata_job_config
        )

        # Wait for the job to complete
        load_job.result()

        # Check the result for each file
        app_metadata_details_table = bq_client.get_table(app_metadata_table_ref)
        logger.info(f"Loaded {app_metadata_details_table.num_rows} rows from {file_name} into {dataset_id}:app_metadata.")

        # Clean up: delete the temporary Parquet file if needed (optional)
        # gcs_client.bucket(bucket_name).delete_blob(f"temp_{file_name}")

logger.info("All files processed successfully.")


Streaming output truncated to the last 5000 lines.
2024-09-26 13:23:23,778 - INFO - Loaded 134 rows from 2024_09_25__1112488968.parquet into app_metadata:app_metadata.
INFO:__main__:Loaded 134 rows from 2024_09_25__1112488968.parquet into app_metadata:app_metadata.
2024-09-26 13:23:23,785 - INFO - Starting load job for file: 2024_09_25__1112861301.parquet
INFO:__main__:Starting load job for file: 2024_09_25__1112861301.parquet
2024-09-26 13:23:27,730 - INFO - Loaded 135 rows from 2024_09_25__1112861301.parquet into app_metadata:app_metadata.
INFO:__main__:Loaded 135 rows from 2024_09_25__1112861301.parquet into app_metadata:app_metadata.
2024-09-26 13:23:27,734 - INFO - Starting load job for file: 2024_09_25__1113002956.parquet
INFO:__main__:Starting load job for file: 2024_09_25__1113002956.parquet
2024-09-26 13:23:34,001 - INFO - Loaded 136 rows from 2024_09_25__1113002956.parquet into app_metadata:app_metadata.
INFO:__main__:Loaded 136 rows from 2024_09_25__1113002956.parquet into a

Forbidden: 403 Quota exceeded: Your table exceeded quota for imports or query appends per table. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas; reason: quotaExceeded, location: load_job_per_table.long, message: Quota exceeded: Your table exceeded quota for imports or query appends per table. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

In [24]:
import re
import pandas as pd

# Define the function to handle log lines and extract parquet file names
def extract_parquet_filenames(log_lines):
    file_names = []
    pattern = r"(\d{4}_\d{2}_\d{2}__\d+\.parquet)"  # Regex pattern for parquet file names

    for log_line in log_lines:
        log_line = log_line.strip()  # Remove unnecessary leading/trailing spaces
        match = re.search(pattern, log_line)
        if match:
            file_names.append(match.group(1))

    return file_names

# Example: If logs are stored in a large string (could also come from a file)
logs = """
INFO 2024-09-26T15:13:40.758461Z Loaded 1219 rows from 2024_09_25__458451517.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:13:45.774428Z Loaded 1220 rows from 2024_09_25__458507285.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:13:55.825593Z Loaded 1221 rows from 2024_09_25__459313476.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:00.830542Z Loaded 1222 rows from 2024_09_25__459749670.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:06.167982Z Loaded 1223 rows from 2024_09_25__460177396.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:11.691050Z Loaded 1224 rows from 2024_09_25__462638897.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:17.381537Z Loaded 1225 rows from 2024_09_25__466446054.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:22.701078Z Loaded 1226 rows from 2024_09_25__467225713.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:28.594891Z Loaded 1227 rows from 2024_09_25__467329677.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:32.953231Z Loaded 1228 rows from 2024_09_25__467398577.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:38.482525Z Loaded 1229 rows from 2024_09_25__467703220.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:43.072552Z Loaded 1230 rows from 2024_09_25__468368751.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:49.179216Z Loaded 1231 rows from 2024_09_25__468996152.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:14:54.236732Z Loaded 1232 rows from 2024_09_25__469284907.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:00.036224Z Loaded 1233 rows from 2024_09_25__469337564.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:05.704815Z Loaded 1234 rows from 2024_09_25__470412147.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:12.316813Z Loaded 1235 rows from 2024_09_25__472014516.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:18.351286Z Loaded 1236 rows from 2024_09_25__472515452.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:24.617441Z Loaded 1237 rows from 2024_09_25__472530750.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:37.469730Z Loaded 1238 rows from 2024_09_25__472969317.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:46.216952Z Loaded 1239 rows from 2024_09_25__472971390.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:51.169016Z Loaded 1240 rows from 2024_09_25__474121500.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:15:57.358437Z Loaded 1241 rows from 2024_09_25__474839398.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:04.566419Z Loaded 1242 rows from 2024_09_25__475249619.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:10.267019Z Loaded 1243 rows from 2024_09_25__476621639.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:16.123991Z Loaded 1244 rows from 2024_09_25__476837843.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:22.079716Z Loaded 1245 rows from 2024_09_25__477128284.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:27.947648Z Loaded 1246 rows from 2024_09_25__477537958.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:32.585691Z Loaded 1247 rows from 2024_09_25__478126577.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:48.512889Z Loaded 1248 rows from 2024_09_25__478462401.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:16:55.562550Z Loaded 1249 rows from 2024_09_25__478610069.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:00.870383Z Loaded 1250 rows from 2024_09_25__480883488.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:06.296272Z Loaded 1251 rows from 2024_09_25__482066631.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:10.367628Z Loaded 1252 rows from 2024_09_25__482303584.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:16.882186Z Loaded 1253 rows from 2024_09_25__483024869.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:22.359713Z Loaded 1254 rows from 2024_09_25__483030834.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:28.258008Z Loaded 1255 rows from 2024_09_25__483241160.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:33.200896Z Loaded 1256 rows from 2024_09_25__485015785.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:37.768844Z Loaded 1257 rows from 2024_09_25__485126024.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:46.043728Z Loaded 1258 rows from 2024_09_25__485357017.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:50.393043Z Loaded 1259 rows from 2024_09_25__487608658.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:17:54.358871Z Loaded 1260 rows from 2024_09_25__487845964.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:00.394452Z Loaded 1261 rows from 2024_09_25__487922291.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:06.538071Z Loaded 1262 rows from 2024_09_25__488818252.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:13.400914Z Loaded 1263 rows from 2024_09_25__489506212.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:18.813111Z Loaded 1264 rows from 2024_09_25__490163532.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:24.409185Z Loaded 1265 rows from 2024_09_25__490179405.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:30.069960Z Loaded 1266 rows from 2024_09_25__491185234.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:36.468964Z Loaded 1267 rows from 2024_09_25__491299921.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:42.216156Z Loaded 1268 rows from 2024_09_25__492076105.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:48.428957Z Loaded 1269 rows from 2024_09_25__492386475.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:52.751625Z Loaded 1270 rows from 2024_09_25__493145008.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:18:56.780143Z Loaded 1271 rows from 2024_09_25__493157418.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:05.905933Z Loaded 1272 rows from 2024_09_25__495369748.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:12.440031Z Loaded 1273 rows from 2024_09_25__497387361.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:18.284268Z Loaded 1274 rows from 2024_09_25__497716362.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:25.065499Z Loaded 1275 rows from 2024_09_25__497880658.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:29.555327Z Loaded 1276 rows from 2024_09_25__498151501.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:34.333623Z Loaded 1277 rows from 2024_09_25__498732510.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:41.059592Z Loaded 1278 rows from 2024_09_25__499470113.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:45.770018Z Loaded 1279 rows from 2024_09_25__499648757.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:51.305915Z Loaded 1280 rows from 2024_09_25__500003565.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:19:57.093816Z Loaded 1281 rows from 2024_09_25__500243933.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:01.975841Z Loaded 1282 rows from 2024_09_25__500361317.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:09.174716Z Loaded 1283 rows from 2024_09_25__501324867.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:14.987275Z Loaded 1284 rows from 2024_09_25__502411889.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:20.223184Z Loaded 1285 rows from 2024_09_25__503451073.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:26.550843Z Loaded 1286 rows from 2024_09_25__503641946.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:31.496821Z Loaded 1287 rows from 2024_09_25__504065732.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:35.970921Z Loaded 1288 rows from 2024_09_25__506076565.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:41.785157Z Loaded 1289 rows from 2024_09_25__506127782.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:46.294242Z Loaded 1290 rows from 2024_09_25__506508166.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:52.700785Z Loaded 1291 rows from 2024_09_25__506627515.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:20:59.511701Z Loaded 1292 rows from 2024_09_25__507097008.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:10.435755Z Loaded 1293 rows from 2024_09_25__507407813.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:16.109153Z Loaded 1294 rows from 2024_09_25__507874739.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:22.827505Z Loaded 1295 rows from 2024_09_25__509576822.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:29.327848Z Loaded 1296 rows from 2024_09_25__510855668.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:34.326409Z Loaded 1297 rows from 2024_09_25__512153051.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:38.849615Z Loaded 1298 rows from 2024_09_25__513131251.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:44.391114Z Loaded 1299 rows from 2024_09_25__513135722.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:48.713620Z Loaded 1300 rows from 2024_09_25__514345557.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:21:53.766768Z Loaded 1301 rows from 2024_09_25__514374715.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:00.974283Z Loaded 1302 rows from 2024_09_25__514485964.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:07.290455Z Loaded 1303 rows from 2024_09_25__514779369.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:12.112596Z Loaded 1304 rows from 2024_09_25__516000464.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:17.407854Z Loaded 1305 rows from 2024_09_25__516047066.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:22.237276Z Loaded 1306 rows from 2024_09_25__517539894.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:28.594344Z Loaded 1307 rows from 2024_09_25__518086179.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:34.602122Z Loaded 1308 rows from 2024_09_25__518684914.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:40.637175Z Loaded 1309 rows from 2024_09_25__519625336.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:47.067035Z Loaded 1310 rows from 2024_09_25__519817714.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:54.188452Z Loaded 1311 rows from 2024_09_25__521227008.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:22:59.379667Z Loaded 1312 rows from 2024_09_25__521487551.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:07.064318Z Loaded 1313 rows from 2024_09_25__522826277.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:12.691431Z Loaded 1314 rows from 2024_09_25__523366295.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:19.448516Z Loaded 1315 rows from 2024_09_25__523419125.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:25.083470Z Loaded 1316 rows from 2024_09_25__524331467.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:32.782056Z Loaded 1317 rows from 2024_09_25__525049909.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:39.917798Z Loaded 1318 rows from 2024_09_25__525170908.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:47.153934Z Loaded 1319 rows from 2024_09_25__525299323.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:52.698351Z Loaded 1320 rows from 2024_09_25__526758956.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:23:57.842447Z Loaded 1321 rows from 2024_09_25__526791870.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:05.903880Z Loaded 1322 rows from 2024_09_25__527050465.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:13.447164Z Loaded 1323 rows from 2024_09_25__528246186.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:19.308049Z Loaded 1324 rows from 2024_09_25__529092671.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:24.960386Z Loaded 1325 rows from 2024_09_25__529379082.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:31.316114Z Loaded 1326 rows from 2024_09_25__529479190.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:37.634311Z Loaded 1327 rows from 2024_09_25__529996768.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:45.818875Z Loaded 1328 rows from 2024_09_25__530168168.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:51.535722Z Loaded 1329 rows from 2024_09_25__530230552.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:24:56.851034Z Loaded 1330 rows from 2024_09_25__530829008.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:03.496912Z Loaded 1331 rows from 2024_09_25__531204555.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:08.908263Z Loaded 1332 rows from 2024_09_25__532331192.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:15.009009Z Loaded 1333 rows from 2024_09_25__533668053.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:21.287347Z Loaded 1334 rows from 2024_09_25__534174796.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:26.360445Z Loaded 1335 rows from 2024_09_25__534343507.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:31.830830Z Loaded 1336 rows from 2024_09_25__535509415.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:37.231814Z Loaded 1337 rows from 2024_09_25__535886823.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:43.446229Z Loaded 1338 rows from 2024_09_25__535960556.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:48.570583Z Loaded 1339 rows from 2024_09_25__536496020.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:25:54.383810Z Loaded 1340 rows from 2024_09_25__536496894.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:00.132892Z Loaded 1341 rows from 2024_09_25__537997638.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:08.379039Z Loaded 1342 rows from 2024_09_25__538212549.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:13.933635Z Loaded 1343 rows from 2024_09_25__539418490.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:19.274385Z Loaded 1344 rows from 2024_09_25__540518599.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:27.110889Z Loaded 1345 rows from 2024_09_25__541164041.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:32.844499Z Loaded 1346 rows from 2024_09_25__541831736.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:37.397916Z Loaded 1347 rows from 2024_09_25__541933937.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:44.759389Z Loaded 1348 rows from 2024_09_25__542557212.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:49.759096Z Loaded 1349 rows from 2024_09_25__542613198.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:26:56.557227Z Loaded 1350 rows from 2024_09_25__543090051.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:02.119833Z Loaded 1351 rows from 2024_09_25__543929148.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:08.174800Z Loaded 1352 rows from 2024_09_25__544007664.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:14.173429Z Loaded 1353 rows from 2024_09_25__544098932.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:20.021654Z Loaded 1354 rows from 2024_09_25__545519333.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:27.401539Z Loaded 1355 rows from 2024_09_25__545599256.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:34.662477Z Loaded 1356 rows from 2024_09_25__546473125.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:40.003819Z Loaded 1357 rows from 2024_09_25__546505307.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:46.116992Z Loaded 1358 rows from 2024_09_25__547313550.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:51.789182Z Loaded 1359 rows from 2024_09_25__547436543.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:27:57.290087Z Loaded 1360 rows from 2024_09_25__547702041.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:02.866205Z Loaded 1361 rows from 2024_09_25__547951480.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:08.860388Z Loaded 1362 rows from 2024_09_25__548942104.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:15.215547Z Loaded 1363 rows from 2024_09_25__549493839.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:20.578588Z Loaded 1364 rows from 2024_09_25__549867299.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:28.049025Z Loaded 1365 rows from 2024_09_25__550039208.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:34.583911Z Loaded 1366 rows from 2024_09_25__550221096.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:40.938556Z Loaded 1367 rows from 2024_09_25__552602056.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:47.864767Z Loaded 1368 rows from 2024_09_25__553602064.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:28:55.642865Z Loaded 1369 rows from 2024_09_25__553834731.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:00.763634Z Loaded 1370 rows from 2024_09_25__555063314.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:11.208366Z Loaded 1371 rows from 2024_09_25__555376968.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:16.496258Z Loaded 1372 rows from 2024_09_25__555415515.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:24.322489Z Loaded 1373 rows from 2024_09_25__556540446.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:29.764251Z Loaded 1374 rows from 2024_09_25__556653197.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:34.021903Z Loaded 1375 rows from 2024_09_25__557405467.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:40.445300Z Loaded 1376 rows from 2024_09_25__558020762.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:45.614681Z Loaded 1377 rows from 2024_09_25__558525346.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:50.531479Z Loaded 1378 rows from 2024_09_25__559977766.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:29:56.649902Z Loaded 1379 rows from 2024_09_25__560068813.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:30:04.615895Z Loaded 1380 rows from 2024_09_25__561311442.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:30:09.830822Z Loaded 1381 rows from 2024_09_25__561352659.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:30:16.427824Z Loaded 1382 rows from 2024_09_25__561625752.parquet into app_metadata:app_metadata.
INFO 2024-09-26T15:30:21.978673Z Loaded 1383 rows from 2024_09_25__561930308.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:13:56.325661Z Loaded 1384 rows from 2024_09_25__1000897885.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:14:03.717668Z Loaded 1385 rows from 2024_09_25__1010311475.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:14:11.538321Z Loaded 1386 rows from 2024_09_25__1010729050.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:14:17.518718Z Loaded 1387 rows from 2024_09_25__1011116985.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:14:35.362891Z Loaded 1388 rows from 2024_09_25__1011476555.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:14:40.555143Z Loaded 1389 rows from 2024_09_25__1011479119.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:14:46.630113Z Loaded 1390 rows from 2024_09_25__1013961111.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:14:52.904284Z Loaded 1391 rows from 2024_09_25__1014050472.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:15:03.902313Z Loaded 1392 rows from 2024_09_25__1015463791.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:16:43.438783Z Loaded 1393 rows from 2024_09_25__1015997735.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:16:48.561114Z Loaded 1394 rows from 2024_09_25__1016323413.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:16:54.555299Z Loaded 1395 rows from 2024_09_25__1016788331.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:17:00.318882Z Loaded 1396 rows from 2024_09_25__1017353772.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:17:06.203693Z Loaded 1397 rows from 2024_09_25__1017354785.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:17:12.779962Z Loaded 1398 rows from 2024_09_25__1017447032.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:17:19.394731Z Loaded 1399 rows from 2024_09_25__1017492454.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:17:29.964988Z Loaded 1400 rows from 2024_09_25__1017800835.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:17:57.303313Z Loaded 1401 rows from 2024_09_25__1020271781.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:04.762979Z Loaded 1402 rows from 2024_09_25__1020486527.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:10.235627Z Loaded 1403 rows from 2024_09_25__1021085625.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:16.035232Z Loaded 1404 rows from 2024_09_25__1023499075.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:22.924Z Loaded 1405 rows from 2024_09_25__1024818709.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:31.475641Z Loaded 1406 rows from 2024_09_25__1026211399.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:48.092544Z Loaded 1407 rows from 2024_09_25__1027517556.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:53.473894Z Loaded 1408 rows from 2024_09_25__1028362533.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:18:58.418124Z Loaded 1409 rows from 2024_09_25__1028592642.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:04.225228Z Loaded 1410 rows from 2024_09_25__1029925317.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:11.103654Z Loaded 1411 rows from 2024_09_25__1030379009.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:16.680429Z Loaded 1412 rows from 2024_09_25__1030595027.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:22.391125Z Loaded 1413 rows from 2024_09_25__1030739155.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:28.242935Z Loaded 1414 rows from 2024_09_25__1030894438.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:33.769074Z Loaded 1415 rows from 2024_09_25__1030896681.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:39.382469Z Loaded 1416 rows from 2024_09_25__1031002863.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:44.641335Z Loaded 1417 rows from 2024_09_25__1031364004.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:49.856576Z Loaded 1418 rows from 2024_09_25__1031903929.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:19:55.626605Z Loaded 1419 rows from 2024_09_25__1033342465.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:01.248216Z Loaded 1420 rows from 2024_09_25__1033598731.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:07.462292Z Loaded 1421 rows from 2024_09_25__1033868403.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:12.833829Z Loaded 1422 rows from 2024_09_25__1034807605.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:18.394028Z Loaded 1423 rows from 2024_09_25__1035430719.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:24.417645Z Loaded 1424 rows from 2024_09_25__1035712810.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:31.205030Z Loaded 1425 rows from 2024_09_25__1036121038.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:37.477266Z Loaded 1426 rows from 2024_09_25__1036175909.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:42.667016Z Loaded 1427 rows from 2024_09_25__1038369065.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:48.185266Z Loaded 1428 rows from 2024_09_25__1038653883.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:20:53.761595Z Loaded 1429 rows from 2024_09_25__1040093707.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:00.371601Z Loaded 1430 rows from 2024_09_25__1041410548.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:04.703649Z Loaded 1431 rows from 2024_09_25__1041727137.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:17.641117Z Loaded 1432 rows from 2024_09_25__1043246998.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:24.062576Z Loaded 1433 rows from 2024_09_25__1044456188.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:29.783996Z Loaded 1434 rows from 2024_09_25__1044702761.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:37.152365Z Loaded 1435 rows from 2024_09_25__1046172976.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:42.817253Z Loaded 1436 rows from 2024_09_25__1046583900.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:48.659232Z Loaded 1437 rows from 2024_09_25__1048524688.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:21:54.647890Z Loaded 1438 rows from 2024_09_25__1048775759.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:22:00.547787Z Loaded 1439 rows from 2024_09_25__1050576032.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:22:05.702567Z Loaded 1440 rows from 2024_09_25__1052879175.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:22:11.501173Z Loaded 1441 rows from 2024_09_25__1053012308.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:22:16.705355Z Loaded 1442 rows from 2024_09_25__1054033219.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:22:22.106307Z Loaded 1443 rows from 2024_09_25__1056652614.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:22:27.414071Z Loaded 1444 rows from 2024_09_25__1056813463.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:43:59.365337Z Loaded 1445 rows from 2024_09_25__1000897885.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:04.445566Z Loaded 1446 rows from 2024_09_25__1010311475.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:08.463824Z Loaded 1447 rows from 2024_09_25__1010729050.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:14.824476Z Loaded 1448 rows from 2024_09_25__1011116985.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:19.819163Z Loaded 1449 rows from 2024_09_25__1011476555.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:24.680282Z Loaded 1450 rows from 2024_09_25__1011479119.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:31.929641Z Loaded 1451 rows from 2024_09_25__1013961111.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:37.518375Z Loaded 1452 rows from 2024_09_25__1014050472.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:43.848901Z Loaded 1453 rows from 2024_09_25__1015463791.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:48.377894Z Loaded 1454 rows from 2024_09_25__1015997735.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:44:53.391882Z Loaded 1455 rows from 2024_09_25__1016323413.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:00.456717Z Loaded 1456 rows from 2024_09_25__1016788331.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:05.946092Z Loaded 1457 rows from 2024_09_25__1017353772.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:12.627232Z Loaded 1458 rows from 2024_09_25__1017354785.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:19.314437Z Loaded 1459 rows from 2024_09_25__1017447032.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:24.741374Z Loaded 1460 rows from 2024_09_25__1017492454.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:30.552418Z Loaded 1461 rows from 2024_09_25__1017800835.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:37.183209Z Loaded 1462 rows from 2024_09_25__1020271781.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:42.746918Z Loaded 1463 rows from 2024_09_25__1020486527.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:48.337729Z Loaded 1464 rows from 2024_09_25__1021085625.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:54.744996Z Loaded 1465 rows from 2024_09_25__1023499075.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:45:59.162327Z Loaded 1466 rows from 2024_09_25__1024818709.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:46:04.969876Z Loaded 1467 rows from 2024_09_25__1026211399.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:46:10.352434Z Loaded 1468 rows from 2024_09_25__1027517556.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:46:16.814881Z Loaded 1469 rows from 2024_09_25__1028362533.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:46:21.021303Z Loaded 1470 rows from 2024_09_25__1028592642.parquet into app_metadata:app_metadata.
INFO 2024-09-26T16:46:25.893422Z Loaded 1471 rows from 2024_09_25__1029925317.parquet into app_metadata:app_metadata.

"""

# Split the logs into individual lines
log_lines = logs.splitlines()

# Call the function to extract parquet file names
file_names = extract_parquet_filenames(log_lines)

# Convert the list of extracted file names to a pandas DataFrame
files_to_skip = pd.DataFrame(file_names, columns=['Parquet File Name'])



In [29]:
files_to_skip.tail()

,Parquet File Name
221,2024_09_25__1052879175.parquet
222,2024_09_25__1053012308.parquet
223,2024_09_25__1054033219.parquet
224,2024_09_25__1056652614.parquet
225,2024_09_25__1056813463.parquet


In [30]:
import logging
from google.cloud import bigquery, storage
import pandas as pd

# Initialize clients
gcs_client = storage.Client()
bq_client = bigquery.Client()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define your parameters
project_id = "web-scraping-2024"
dataset_id = "app_metadata"
bucket_name = "app_metadata_top_chart"
app_metadata_dir_prefix = ""

def list_parquet_files(bucket_name, prefix):
    """List all parquet files in a GCS bucket with a given prefix."""
    bucket = gcs_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.parquet')]

def update_table_schema_to_string(table_ref):
    """Update all fields in the table schema to STRING."""
    table = bq_client.get_table(table_ref)
    logger.info(f"Current schema: {table.schema}")

    new_schema = []
    for field in table.schema:
        new_schema.append(bigquery.SchemaField(field.name, "STRING"))  # Change all fields to STRING

    # Update the table with the modified schema
    table.schema = new_schema
    updated_table = bq_client.update_table(table, ["schema"])  # Update only the schema
    logger.info(f"Updated table {updated_table.project}.{updated_table.dataset_id}.{updated_table.table_id} schema to STRING.")

# Load all parquet files
app_metadata_details_files = list_parquet_files(bucket_name, app_metadata_dir_prefix)

if not app_metadata_details_files:
    logger.warning("No parquet files found.")
else:
    app_metadata_table_ref = bq_client.dataset(dataset_id).table("app_metadata")

    # Update the schema to modify all fields to STRING
    update_table_schema_to_string(app_metadata_table_ref)

    app_metadata_job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,  # Parquet file format
        autodetect=True
    )

    for file_name in app_metadata_details_files:
        # Check if the file is in the skip list
        if file_name in files_to_skip:
            logger.info(f"Skipping file: {file_name}")
            continue  # Skip files that are in the skip list

        logger.info(f"Starting load job for file: {file_name}")

        # Load the Parquet file into a DataFrame to convert all data to STRING
        df = pd.read_parquet(f"gs://{bucket_name}/{file_name}")

        # Convert all columns to STRING
        df = df.astype(str)

        # Save the modified DataFrame back to a temporary Parquet file
        temp_parquet_path = f"gs://{bucket_name}/temp_{file_name}"
        df.to_parquet(temp_parquet_path)

        # Start the load job for each file
        load_job = bq_client.load_table_from_uri(
            temp_parquet_path,
            app_metadata_table_ref,
            job_config=app_metadata_job_config
        )

        # Wait for the job to complete
        load_job.result()

        # Check the result for each file
        app_metadata_details_table = bq_client.get_table(app_metadata_table_ref)
        logger.info(f"Loaded {app_metadata_details_table.num_rows} rows from {file_name} into {dataset_id}:app_metadata.")

        # Clean up: delete the temporary Parquet file if needed (optional)
        # gcs_client.bucket(bucket_name).delete_blob(f"temp_{file_name}")

logger.info("All files processed successfully.")


2024-09-26 16:43:52,479 - INFO - Current schema: [SchemaField('scraping_url', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('scraping_timestamp', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('scraping_url_redirect', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('title', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('subtitle', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('avg_rating', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('number_of_ratings', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('developer_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('developer_url', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('size', 'STRING', 'NULLABLE', None, None, (), None)]
INFO:__main__:Current schema: [SchemaField('scraping_url', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('scraping_timestamp', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('scraping_url_redirect'

KeyboardInterrupt: 